In [1]:
import pandas as pd
import datetime as dt
import locale
import holidays

In [2]:
# --- Configurar los días festivos para Colombia para 2025:
co_holidays = holidays.CountryHoliday('CO', years=[2025])

In [3]:
ruta ="/home/donsson/proyectos/BIOMETRICO/dataset/Biometrico BQ 09.25.xls"
df = pd.read_excel(ruta)
mes = 9
sucursal =  "BQ"


WARNING *** file size (184313) not 512 + multiple of sector size (512)


In [4]:
df.isnull().sum()

Número                0
Nombre                0
Tiempo                0
Estado              305
Dispositivos          0
Tipo de Registro      0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Número            752 non-null    int64 
 1   Nombre            752 non-null    object
 2   Tiempo            752 non-null    object
 3   Estado            447 non-null    object
 4   Dispositivos      752 non-null    object
 5   Tipo de Registro  752 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 35.4+ KB


In [6]:
# Manejar por buenas practicas titulos en minuscula y sin espacios en blanco
new_name =[] 
for column in df.columns:
    minus = column.lower()
    replace = minus.replace(" ","_")
    new_name.append(replace)

df.columns = new_name


df.duplicated().sum()


np.int64(0)

In [7]:
#cambia el nombre de la columna de numero a ID del trabajador
df.rename(columns={'número': 'id'}, inplace=True) 

In [8]:
# Reemplaza el espacio no-breaking por un espacio normal
df['tiempo'] = df['tiempo'].str.replace('\xa0', ' ', regex=False)  

# Reemplaza a.m a AM para que datetime lo lea 
df['tiempo'] = df['tiempo'].str.replace('a. m.', 'AM', regex=False, case=False) 

# Reemplaza p.m a PM para que datetime lo lea 
df['tiempo'] = df['tiempo'].str.replace('p. m.', 'PM', regex=False, case=False) 



df

,id,nombre,tiempo,estado,dispositivos,tipo_de_registro
0,79350377,JAIME CAÑON,1/08/2025 8:07:57 AM,Entrada,barranquilla,0
1,79350377,JAIME CAÑON,1/08/2025 5:05:12 PM,Salida,barranquilla,0
2,79350377,JAIME CAÑON,4/08/2025 2:03:52 PM,Entrada,barranquilla,0
3,79350377,JAIME CAÑON,4/08/2025 5:04:48 PM,Salida,barranquilla,0
4,79350377,JAIME CAÑON,5/08/2025 7:58:02 AM,Entrada,barranquilla,0
...,...,...,...,...,...,...
747,104575797,David de Jesus Fernandez Tette,30/09/2025 7:12:02 AM,Entrada,barranquilla,0
748,104575797,David de Jesus Fernandez Tette,30/09/2025 12:23:55 PM,NaN,barranquilla,0
749,104575797,David de Jesus Fernandez Tette,30/09/2025 1:27:32 PM,NaN,barranquilla,0
750,104575797,David de Jesus Fernandez Tette,30/09/2025 4:39:14 PM,NaN,barranquilla,0


In [9]:
df['tiempo'] = pd.to_datetime(df['tiempo'], dayfirst=True, errors='coerce')

# Crear columnas adicionales
df['fecha'] = df['tiempo'].dt.date
df['hora'] = df['tiempo'].dt.time
df['mes'] = df['tiempo'].dt.month


In [10]:
# Extraer la hora (en formato 24 horas)
df['hora_entera'] = df['tiempo'].dt.hour

# Extraer los minutos
df['minutos_enteros'] = df['tiempo'].dt.minute

# Calcular la fracción de la hora (como en Excel)
df['fraccion_dia_hora'] = (df['hora_entera'] + df['minutos_enteros'] / 60) / 24


In [11]:
# Crear una lista de nombres de días de la semana en español para mapear
dias_semana_espanol = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]

# Extraer el número del día de la semana (0=lunes, 6=domingo)
#    Lo aplicamos a la parte de la fecha de la columna 'tiempo'
df['dia_Numero'] = df['tiempo'].dt.weekday

# Mapear el número a su nombre correspondiente en español
df['dia'] = df['dia_Numero'].map(lambda x: dias_semana_espanol[x])


# Busqueda de dias festivos
df['festivo'] = df['fecha'].apply(lambda x: x in co_holidays)


df = df[df["mes"]==mes]


fecha_inicio = df["fecha"].min()
fecha_fin = df["fecha"].max()


df["nombre"] = df["nombre"].str.upper()



empleados = {
    "ANDRES FELIPE ORDOÑEZ GREGORY": "Mostrador",
    "CARLOS ANDRES CABYALES PRADO": "Mostrador",
    "CLAUDIA XIMENA ORTIZ CAMACHO": "Mostrador",
    "DAVID DE JESUS FERNANDEZ TETTE": "Mostrador",
    "DIEGO ANDRES CARDONA VILLA": "Mostrador",
    "EDINSON JAVIER GARCIA MURILLO": "Mostrador",
    "EDISON MIGUEL CIFUENTES OSORIO": "Mostrador",
    "EDWIN FERNANDO COLLAZOS BARRETO": "Mostrador",
    "EVERTH DE JESUS OQUENDO JIMENEZ": "Externo",
    "GERMAN ANTONIO MORALES CORDENAS": "Mostrador",
    "JAIME CAÑON": "Externo",
    "JAIME ANDRES CABALLERO CABALLERO": "Mostrador",
    "JAIME ANDRES DUARTE MORA": "Externo",
    "JAIME SEBASTIAN RODRIGUEZ RAMOS": "Mostrador",
    "JEYSSON FABIAN BLANCO RODRIGUEZ": "Mostrador",
    "JHON MULATO": "Mostrador",
    "JUAN DAVID SOGAMOSO TIRADO": "Mostrador",
    "JUAN JOSE OSPINA CARDONA": "Mostrador",
    "KENER DAVID SANTAMARIA": "Mostrador",
    "MARCO MAURICIO MEJIA RAMIREZ": "Mostrador",
    "MARIA CAMILA ARGUELLO BERMUDEZ": "Mostrador",
    "MIGUEL NIEBLES": "Externo",
    "NATALI ESPINAL TRUJILLO": "Mostrador",
    "NEYDER CRESPO": "Mostrador",
    "OMAR YADIR HERNANDEZ PULGARIN": "Externo",
    "PATRICIA NUÑEZ VALENCIA": "Externo",
    "SAID DAVID VARGAS LOPEZ": "Mostrador",
    "SANDRA PATICIA RAMIREZ CORREDOR": "Mostrador",
    "SANDRA ROCIO FONSECA MOYANO": "Mostrador",
    "VICTOR ALEJANDRO GONZALEZ RODRIGUEZ": "Mostrador",
    "WILSON ALEJANDRO PEÑAEZ PEREZ": "Mostrador",
    "YADI ALMEETH RODRIGUEZ JOTA": "Mostrador",
    "YERITZA MARIA PALMA BARRERA": "Mostrador",
    "YURI NATALIA OJANA PARRA": "Mostrador",
    "AURELIYASTRA SERI FAAS ROCHA": "Mostrador",
    "JUAN ESTEBAN TORRES ROCHA": "Mostrador",
    "YONEINKER JOSE GONZALEZ HERNANDEZ": "Mostrador",
    "YEIS MUÑOZ": "Externo"
}

df["cargo"] = df["nombre"].map(empleados)
df["nombre"].unique()

/tmp/ipykernel_24874/2473898982.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["nombre"] = df["nombre"].str.upper()
/tmp/ipykernel_24874/2473898982.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cargo"] = df["nombre"].map(empleados)


array(['JAIME CAÑON', 'NEYDER CRESPO', 'OMAR JOSE CARRILLO BERDEJO',
       'MIGUEL NIEBLES', 'RICARDO ANDRES MEJIA HERNADEZ',
       'KENER DAVID SANTAMARIA', 'DAVID DE JESUS FERNANDEZ TETTE'],
      dtype=object)

In [12]:
df.head()

,id,nombre,tiempo,estado,dispositivos,tipo_de_registro,fecha,hora,mes,hora_entera,minutos_enteros,fraccion_dia_hora,dia_Numero,dia,festivo,cargo
15,79350377,JAIME CAÑON,2025-09-01 07:06:28,Entrada,barranquilla,0,2025-09-01,07:06:28,9,7,6,0.295833,0,lunes,False,Externo
16,79350377,JAIME CAÑON,2025-09-01 17:05:33,Salida,barranquilla,0,2025-09-01,17:05:33,9,17,5,0.711806,0,lunes,False,Externo
17,79350377,JAIME CAÑON,2025-09-02 08:01:26,Entrada,barranquilla,0,2025-09-02,08:01:26,9,8,1,0.334028,1,martes,False,Externo
18,79350377,JAIME CAÑON,2025-09-05 08:02:20,Entrada,barranquilla,0,2025-09-05,08:02:20,9,8,2,0.334722,4,viernes,False,Externo
19,79350377,JAIME CAÑON,2025-09-17 17:05:38,Salida,barranquilla,0,2025-09-17,17:05:38,9,17,5,0.711806,2,miércoles,False,Externo


In [13]:
# Agrupar por persona y día, y obtener la primera y última marca
horarios = df.groupby(['nombre', 'fecha','cargo'])['tiempo'].agg(['min', 'max']).reset_index()
horarios.rename(columns={'min': 'hora_entrada', 'max': 'hora_salida'}, inplace=True)

horarios['horas_trabajadas'] = (horarios['hora_salida'] - horarios['hora_entrada']).dt.total_seconds() / 3600


horarios['fecha'] = pd.to_datetime(horarios['fecha'], errors='coerce')
horarios['mes'] = horarios['fecha'].dt.month



In [14]:
resumen = horarios.groupby(['nombre','cargo' ,'mes'])['horas_trabajadas'].sum().reset_index()
resumen.rename(columns={'horas_trabajadas': 'total_horas_efectivas'}, inplace=True)

resumen.head()

,nombre,cargo,mes,total_horas_efectivas
0,DAVID DE JESUS FERNANDEZ TETTE,Mostrador,9,215.257778
1,JAIME CAÑON,Externo,9,9.984722
2,KENER DAVID SANTAMARIA,Mostrador,9,207.446667
3,MIGUEL NIEBLES,Externo,9,70.229167
4,NEYDER CRESPO,Mostrador,9,215.798333


In [15]:
import pandas as pd
import holidays

def calcular_horas_esperadas(fecha_inicio, fecha_fin, tipo_empleado='Mostrador', country_code='CO'):
    """
    Calcula las horas laborales esperadas entre dos fechas, excluyendo domingos y feriados.
    Maneja diferentes horarios según el tipo de empleado.

    Args:
        fecha_inicio (str): Fecha de inicio en formato 'YYYY-MM-DD'.
        fecha_fin (str): Fecha de fin en formato 'YYYY-MM-DD'.
        tipo_empleado (str): Tipo de empleado ('Mostrador' o 'Externo').
        country_code (str): Código del país para los feriados (ej. 'CO' para Colombia).

    Returns:
        float: Total de horas esperadas.
    """
    
    # Normalizar tipo de empleado
    tipo_empleado_lower = tipo_empleado.lower().strip()
    
    # Definir horarios según tipo de empleado
    if tipo_empleado_lower == 'mostrador':
        # Mostrador: Lunes a viernes 9 horas, Sábado 4 horas
        horas_lunes_viernes = 9
        horas_sabado = 4
        usar_horario_externo = False
    elif tipo_empleado_lower == 'externo':
        # Vendedor Externo:
        # Lunes: 10 horas (7:00 a.m. - 5:00 p.m.)
        # Martes a viernes: 9 horas (8:00 a.m. - 5:00 p.m.)
        # Sábado: 3 horas (8:00 a.m. - 11:00 a.m.)
        horas_lunes = 10
        horas_martes_viernes = 9
        horas_sabado = 3
        usar_horario_externo = True
    else:
        # Por defecto usar horario de Mostrador
        horas_lunes_viernes = 9
        horas_sabado = 4
        usar_horario_externo = False

    total_horas = 0.0

    # Crear un objeto de la librería holidays para Colombia
    dias_festivos = holidays.CountryHoliday(country_code)

    # Generar un rango de fechas
    fechas = pd.to_datetime(pd.Series(pd.date_range(start=fecha_inicio, end=fecha_fin)))

    for fecha in fechas:
        # Excluir domingos
        if fecha.dayofweek == 6:  # Domingo = 6
            continue

        # Excluir feriados
        if fecha in dias_festivos:
            continue

        # Sumar horas según el tipo de empleado y día de la semana
        if usar_horario_externo:
            if fecha.dayofweek == 0:  # Lunes
                total_horas += horas_lunes
            elif fecha.dayofweek >= 1 and fecha.dayofweek <= 4:  # Martes a viernes
                total_horas += horas_martes_viernes
            elif fecha.dayofweek == 5:  # Sábado
                total_horas += horas_sabado
        else:  # Mostrador
            if fecha.dayofweek >= 0 and fecha.dayofweek <= 4:  # Lunes a viernes
                total_horas += horas_lunes_viernes
            elif fecha.dayofweek == 5:  # Sábado
                total_horas += horas_sabado

    return total_horas


def calcular_horas_para_dataframe(df, fecha_inicio, fecha_fin, columna_cargo='cargo', country_code='CO'):
    """
    Calcula las horas esperadas para cada empleado en un DataFrame según su cargo.
    Agrega la columna 'Horas_Esperadas' al DataFrame original.
    
    Args:
        df (DataFrame): DataFrame con información de empleados.
        fecha_inicio (str): Fecha de inicio en formato 'YYYY-MM-DD'.
        fecha_fin (str): Fecha de fin en formato 'YYYY-MM-DD'.
        columna_cargo (str): Nombre de la columna que contiene el cargo.
        country_code (str): Código del país para los feriados.
    
    Returns:
        DataFrame: DataFrame original con columna adicional 'Horas_Esperadas'.
    """
    
    def determinar_tipo_empleado(cargo):
        """Determina el tipo de empleado basándose en el cargo."""
        cargo_str = str(cargo).lower().strip()
        
        # Buscar si contiene "externo" en el cargo
        if 'Externo' in cargo_str:
            return 'Externo'
        # Buscar si contiene "mostrador" en el cargo
        elif 'Mostrador' in cargo_str:
            return 'Mostrador'
        else:
            # Por defecto, si no coincide con ninguno, usar Mostrador
            return 'Mostrador'
    
    # Calcular horas para cada empleado
    horas_esperadas = []
    
    for cargo in df[columna_cargo]:
        tipo_empleado = determinar_tipo_empleado(cargo)
        horas = calcular_horas_esperadas(fecha_inicio, fecha_fin, tipo_empleado, country_code)
        horas_esperadas.append(horas)
    
    # Agregar columna al DataFrame original
    resumen['Horas_Esperadas'] = horas_esperadas
    
    return df


In [16]:
print(f"Horas esperadas para mostrador :{calcular_horas_esperadas(fecha_inicio, fecha_fin, tipo_empleado='mostrador', country_code='CO')}")
print(f"Horas esperadas para externo :{calcular_horas_esperadas(fecha_inicio, fecha_fin, tipo_empleado='externo', country_code='CO')}")

Horas esperadas para mostrador :214.0
Horas esperadas para externo :215.0


In [17]:
resumen = calcular_horas_para_dataframe(resumen , fecha_inicio = fecha_inicio , fecha_fin= fecha_fin)

In [18]:
resumen.head(10)

,nombre,cargo,mes,total_horas_efectivas,Horas_Esperadas
0,DAVID DE JESUS FERNANDEZ TETTE,Mostrador,9,215.257778,214.0
1,JAIME CAÑON,Externo,9,9.984722,214.0
2,KENER DAVID SANTAMARIA,Mostrador,9,207.446667,214.0
3,MIGUEL NIEBLES,Externo,9,70.229167,214.0
4,NEYDER CRESPO,Mostrador,9,215.798333,214.0


In [19]:
resumen = resumen.copy()
resumen['porcentaje_cumplimiento'] = resumen['total_horas_efectivas'] / resumen['Horas_Esperadas']

In [20]:
import pandas as pd
import numpy as np

    


def calcular_horas_esperadas_por_dia(row):
    # 1. Obtener el día de la semana (0=Lunes, 6=Domingo)
    dia_semana = row["fecha"].weekday()

    # 2. Normalizar cargo
    cargo_lower = str(row["cargo"]).lower().strip()

        # 3. Reglas
    if "externo" in cargo_lower:
        if dia_semana == 0:   # Lunes
            return 10
        elif dia_semana == 5: # Sábado
            return 3
        else:                 # Martes-Viernes
            return 9
    elif "mostrador" in cargo_lower:
        if dia_semana == 5:#Sabado
            return 4
        else:
            return 9


        
# --- 3. Aplicar función para calcular horas esperadas ---
horarios["fecha"] = pd.to_datetime(horarios["fecha"])
horarios["horas_esperadas"] = horarios.apply(calcular_horas_esperadas_por_dia, axis=1)


    
 # --- 4. Detectar errores ---
horarios["trabajo_menos"] = horarios["horas_trabajadas"].round(2) < horarios["horas_esperadas"]



In [21]:
horarios = horarios.drop(columns=["mes"])

In [40]:
horarios.sample(10)

,nombre,fecha,cargo,hora_entrada,hora_salida,horas_trabajadas,horas_esperadas,trabajo_menos
14,DAVID DE JESUS FERNANDEZ TETTE,2025-09-17,Mostrador,2025-09-17 08:24:42,2025-09-17 17:32:31,9.130278,9,False
56,MIGUEL NIEBLES,2025-09-01,Externo,2025-09-01 07:07:22,2025-09-01 17:05:18,9.965556,10,True
7,DAVID DE JESUS FERNANDEZ TETTE,2025-09-09,Mostrador,2025-09-09 07:05:19,2025-09-09 16:35:33,9.503889,9,False
52,KENER DAVID SANTAMARIA,2025-09-26,Mostrador,2025-09-26 07:05:52,2025-09-26 16:09:56,9.067778,9,False
9,DAVID DE JESUS FERNANDEZ TETTE,2025-09-11,Mostrador,2025-09-11 07:10:59,2025-09-11 17:33:20,10.372500,9,False
41,KENER DAVID SANTAMARIA,2025-09-13,Mostrador,2025-09-13 08:05:58,2025-09-13 13:39:22,5.556667,4,False
37,KENER DAVID SANTAMARIA,2025-09-09,Mostrador,2025-09-09 13:38:16,2025-09-09 17:38:03,3.996389,9,True
44,KENER DAVID SANTAMARIA,2025-09-17,Mostrador,2025-09-17 08:34:09,2025-09-17 17:35:44,9.026389,9,False
4,DAVID DE JESUS FERNANDEZ TETTE,2025-09-05,Mostrador,2025-09-05 08:25:03,2025-09-05 17:29:57,9.081667,9,False
35,KENER DAVID SANTAMARIA,2025-09-06,Mostrador,2025-09-06 07:02:19,2025-09-06 11:03:15,4.015556,4,False


In [23]:
with pd.ExcelWriter(f"entregas/reporte_{mes}_{sucursal}.xlsx") as writer:
    resumen.to_excel(writer , sheet_name ="resumen",index=False)
    horarios.to_excel(writer , sheet_name= "diario", index =False)
print("Archivo creado correctamente")

Archivo creado correctamente
